In [27]:
%cd ./filelists/CUB/
!ln -s  ../../../CUB_200_2011/images ./images
%cd ../../
# !ln -s /kaggle/input/miniimagenet/miniImageNet/images ./filelists/miniImagenet/images
# !ln -s /kaggle/input/d/arjun2000ashok/tieredimagenet/tiered_imagenet/ ./filelists/tieredImagenet/images
# !ln -s /kaggle/input/stanford-dogs-dataset/images/Images ./filelists/dogs/Images
# !ln -s /kaggle/input/stanford-cars-dataset/cars_train/cars_train ./filelists/cars/images 
# !ln -s /kaggle/input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images ./filelists/aircrafts/images
# !ln -s /kaggle/input/vggflowers/images ./filelists/flowers/images

/content/gdrive/My Drive/MLRC-models/fsl_ssl/filelists/CUB
/content/gdrive/My Drive/MLRC-models/fsl_ssl


In [1]:
!pip -qqq install tensorboardX

     |████████████████████████████████| 122kB 31.0MB/s 


In [57]:
!python train.py --dataset CUB --train_aug --method baseline --committed

tracking in baseline train: True
USE pre-trained model: False
Checkpoint path: ckpts/CUB/_resnet18_baseline_aug_tracking_lr0.0010
Fresh wandb run
wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
2021-06-06 06:48:37.240701: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
wandb: Tracking run with wandb version 0.10.31
wandb: Syncing run fearless-voice-51
wandb:  View project at https://wandb.ai/multi-input/fsl_ssl
wandb:  View run at https://wandb.ai/multi-input/fsl_ssl/runs/7b45910a
wandb: Run data is saved locally in /content/gdrive/My Drive/MLRC-models/fsl_ssl/wandb/run-20210606_064836-7b45910a
wandb: Run `wandb offline` to turn off syncing.

Epoch 1 | Batch 50/368 | Loss 5.248581 | Acc Cls 0.015000
Error in sys.excepthook:
Traceback (most recent call last):
  File "/usr/lib/python3.7/linecache.py", line 74, in checkcache
    stat = os.stat(fullname)
KeyboardInterrupt

Origin

In [56]:
!nvidia-smi

Failed to initialize NVML: Driver/library version mismatch
